# Background subtraction

This notebook implements the method developed in script Background.ipynb in a loop to handle a batch of images.

It expects .ARW images as input, and generates corresponding .bkg_subtracted.fits files.

In [1]:
import os, glob
import time, datetime
import math

import numpy as np

from scipy import stats

from astropy.io import fits
from astropy.table import Table
from astropy.stats import SigmaClip
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans

from photutils.background import Background2D, MedianBackground, ModeEstimatorBackground

import rawpy
import exifread

from datapath import DATA

## Initialization

In [2]:
# these were determined in script White_light_images as the normalization
# factors that render the smoothest background.
red_norm =  1.34
blue_norm = 1.25

# parameters to control background subtraction
bkg_cell_footprint = (100, 100)
bkg_filter = (11, 11)

bkg_sigma_clip = SigmaClip(sigma=5.)
bkg_kernel = Gaussian2DKernel(x_stddev=1)
bkg_estimator = ModeEstimatorBackground()

In [3]:
# paths
data_dirpath = os.path.join(DATA,'astrophotography_data/Andromeda_2022/135mm16s3200ISO')

input_suffix = '.ARW'
output_suffix = '.bkg_subtracted.fits'

In [4]:
# images to subtract background from
image_list = list(glob.glob(data_dirpath + '/*' + input_suffix))
image_list.sort()

image_list

['/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01262.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01263.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01264.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01265.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01266.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01267.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01268.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01269.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01270.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01271.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC

In [5]:
# read base image - we need this in order to access the camera color matrix
raw = rawpy.imread(image_list[int(len(image_list)/2)]) # mid-point
imarray_base = raw.raw_image_visible.astype(float)

In [6]:
# masks that isolate the RGB pixels
colors_array = raw.raw_colors_visible

red_mask = np.where(colors_array == 0, 1, 0)

green_mask_1 = np.where(colors_array == 1, 1, 0)
green_mask_2 = np.where(colors_array == 3, 1, 0)
green_mask = green_mask_1 | green_mask_2

blue_mask = np.where(colors_array == 2, 1, 0)

## Functions

In [7]:
def subtract_background(imarray, red_norm=1.0, blue_norm=1.0):

    # red_norm and blue_norm are normalization parameters applied to the R and B bands (assume
    # G=1) in order to make the star images as well-behaved as possible, in terms of being 
    # well represented, on average, by the daofind Gaussian. Ideally a different normalization 
    # should be applied to each star, depending on its color index, but this will be left as
    # a possible (but not very likely) future improvement. For now, we assume that an average,
    # frame-wide single normalization should suffice (statistically).
    
    # separate color bands
    red_array = imarray * red_mask
    green_array = imarray * green_mask
    blue_array = imarray * blue_mask
    
    # interpolate over the masked pixels in each band, so the background estimator 
    # is presented with a smooth array entirely filled with valid data
    red_array[red_array == 0.0] = np.nan
    green_array[green_array == 0.0] = np.nan
    blue_array[blue_array == 0.0] = np.nan

    red_array = interpolate_replace_nans(red_array, bkg_kernel)
    green_array = interpolate_replace_nans(green_array, bkg_kernel)
    blue_array = interpolate_replace_nans(blue_array, bkg_kernel)

    red_array[np.isnan(red_array)] = 0.
    green_array[np.isnan(green_array)] = 0.
    blue_array[np.isnan(blue_array)] = 0.
    
    # fit background model to each smoothed-out color band
    red_bkg = Background2D(red_array, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=bkg_sigma_clip, bkg_estimator=bkg_estimator)
    green_bkg = Background2D(green_array, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=bkg_sigma_clip, bkg_estimator=bkg_estimator)
    blue_bkg = Background2D(blue_array, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=bkg_sigma_clip, bkg_estimator=bkg_estimator)

    # subtract background from each masked color array
    subtracted = imarray - red_bkg.background * red_mask - \
                           green_bkg.background * green_mask - \
                           blue_bkg.background * blue_mask

    # after background subtraction, apply color band normalization. This has to be done separately
    # from step above for the background on each band to remain zero on average.
    subtracted = (subtracted * red_mask * red_norm) + \
                 (subtracted * green_mask) + \
                 (subtracted * blue_mask * blue_norm)

    return subtracted

In [8]:
def write_bkg_subtracted(image_name, subtracted):
    today = datetime.datetime.now().ctime()
    
    fits_name = image_name.replace(input_suffix, output_suffix)

    print("Writing bkg-subtracted image: ", fits_name, today)

    # Create FITS 32-bit floating point file with data array
    hdr = fits.Header()
    hdr['DATE'] = today
    hdr['PATH'] = fits_name
    primary_hdu = fits.PrimaryHDU(header=hdr)

    hdu = fits.ImageHDU(subtracted.astype('float32'))

    hdul = fits.HDUList([primary_hdu, hdu])
    hdul.writeto(fits_name, overwrite=True)
    
    return fits_name

## Main loop

In [ ]:
for image_name in image_list:
    raw = rawpy.imread(image_name)
    imarray = raw.raw_image_visible.astype(float)

    subtracted = subtract_background(imarray, red_norm=red_norm, blue_norm=blue_norm)
    
    write_bkg_subtracted(image_name, subtracted)

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01262.bkg_subtracted.fits Tue Sep  5 09:53:27 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01263.bkg_subtracted.fits Tue Sep  5 09:53:36 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01264.bkg_subtracted.fits Tue Sep  5 09:53:46 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01265.bkg_subtracted.fits Tue Sep  5 09:53:56 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01266.bkg_subtracted.fits Tue Sep  5 09:54:05 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01267.bkg_subtracted.fits Tue Sep  5 09:54:15 2023
Writing bkg-subtracted image:  /Users/bu

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01314.bkg_subtracted.fits Tue Sep  5 10:01:41 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01315.bkg_subtracted.fits Tue Sep  5 10:01:50 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01316.bkg_subtracted.fits Tue Sep  5 10:02:00 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01317.bkg_subtracted.fits Tue Sep  5 10:02:09 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01318.bkg_subtracted.fits Tue Sep  5 10:02:18 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01319.bkg_subtracted.fits Tue Sep  5 10:02:28 2023
Writing bkg-subtracted image:  /Users/bu

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01366.bkg_subtracted.fits Tue Sep  5 10:09:53 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01367.bkg_subtracted.fits Tue Sep  5 10:10:03 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01368.bkg_subtracted.fits Tue Sep  5 10:10:12 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01369.bkg_subtracted.fits Tue Sep  5 10:10:22 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01370.bkg_subtracted.fits Tue Sep  5 10:10:31 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01371.bkg_subtracted.fits Tue Sep  5 10:10:41 2023
Writing bkg-subtracted image:  /Users/bu

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01418.bkg_subtracted.fits Tue Sep  5 10:18:07 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01419.bkg_subtracted.fits Tue Sep  5 10:18:17 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01420.bkg_subtracted.fits Tue Sep  5 10:18:26 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01421.bkg_subtracted.fits Tue Sep  5 10:18:36 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01422.bkg_subtracted.fits Tue Sep  5 10:18:45 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01423.bkg_subtracted.fits Tue Sep  5 10:18:54 2023
Writing bkg-subtracted image:  /Users/bu

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01470.bkg_subtracted.fits Tue Sep  5 10:26:21 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01471.bkg_subtracted.fits Tue Sep  5 10:26:30 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01472.bkg_subtracted.fits Tue Sep  5 10:26:40 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01473.bkg_subtracted.fits Tue Sep  5 10:26:49 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01474.bkg_subtracted.fits Tue Sep  5 10:26:59 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01475.bkg_subtracted.fits Tue Sep  5 10:27:08 2023
Writing bkg-subtracted image:  /Users/bu

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01522.bkg_subtracted.fits Tue Sep  5 10:34:33 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01523.bkg_subtracted.fits Tue Sep  5 10:34:42 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01524.bkg_subtracted.fits Tue Sep  5 10:34:52 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01525.bkg_subtracted.fits Tue Sep  5 10:35:01 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01526.bkg_subtracted.fits Tue Sep  5 10:35:11 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01527.bkg_subtracted.fits Tue Sep  5 10:35:20 2023
Writing bkg-subtracted image:  /Users/bu

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01574.bkg_subtracted.fits Tue Sep  5 10:42:45 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01575.bkg_subtracted.fits Tue Sep  5 10:42:55 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01576.bkg_subtracted.fits Tue Sep  5 10:43:04 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01577.bkg_subtracted.fits Tue Sep  5 10:43:14 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01578.bkg_subtracted.fits Tue Sep  5 10:43:23 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01579.bkg_subtracted.fits Tue Sep  5 10:43:32 2023
Writing bkg-subtracted image:  /Users/bu

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01626.bkg_subtracted.fits Tue Sep  5 10:51:00 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01627.bkg_subtracted.fits Tue Sep  5 10:51:09 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01628.bkg_subtracted.fits Tue Sep  5 10:51:19 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01629.bkg_subtracted.fits Tue Sep  5 10:51:28 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01630.bkg_subtracted.fits Tue Sep  5 10:51:38 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01631.bkg_subtracted.fits Tue Sep  5 10:51:48 2023
Writing bkg-subtracted image:  /Users/bu